In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
with open('/content/drive/MyDrive/Next_work_Pred_Dataset.txt', 'r') as file:
    file_data = file.read()

In [19]:
print(file_data)

BACKGROUND AND MOTIVATION
Existing pothole detection systems, predominantly camera-based, face limitations such as weather dependency and restricted camera angles. Our motivation stems from the need to overcome these limitations to enhance road safety and efficiency.
OBJECTIVE
The primary objective of our project is to develop a proactive pothole detection system that empowers drivers with early warnings about approaching hazards. By leveraging geolocation, geofencing, we aim to provide real-time alerts to drivers, potentially preventing accidents and reducing vehicle damage.
IDEA DESCRIPTION 
Our innovative system harnesses geolocation technology for precise user positioning, creating virtual early warning zones (geofencing) around detected potholes. Integration with the Google Maps API enhances functionality, providing real-time traffic data and potentially incorporating user-reported pothole information. This multifaceted approach aims to empower drivers with proactive alerts, enabl

In [20]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_word = False

class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.is_end_of_word = True

    def search(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                return False
            node = node.children[char]
        return node.is_end_of_word


In [21]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [22]:

trie = Trie()
for word in file_data.split():
    trie.insert(word)

tokenizer = Tokenizer()
tokenizer.fit_on_texts([file_data])

input_sequences = []
for sentence in file_data.split('\n'):
    tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]
    for i in range(1, len(tokenized_sentence)):
        input_sequences.append(tokenized_sentence[:i + 1])

input_sequences


[[520, 2],
 [520, 2, 323],
 [79, 6],
 [79, 6, 5],
 [79, 6, 5, 39],
 [79, 6, 5, 39, 238],
 [79, 6, 5, 39, 238, 35],
 [79, 6, 5, 39, 238, 35, 80],
 [79, 6, 5, 39, 238, 35, 80, 521],
 [79, 6, 5, 39, 238, 35, 80, 521, 136],
 [79, 6, 5, 39, 238, 35, 80, 521, 136, 31],
 [79, 6, 5, 39, 238, 35, 80, 521, 136, 31, 27],
 [79, 6, 5, 39, 238, 35, 80, 521, 136, 31, 27, 324],
 [79, 6, 5, 39, 238, 35, 80, 521, 136, 31, 27, 324, 175],
 [79, 6, 5, 39, 238, 35, 80, 521, 136, 31, 27, 324, 175, 2],
 [79, 6, 5, 39, 238, 35, 80, 521, 136, 31, 27, 324, 175, 2, 325],
 [79, 6, 5, 39, 238, 35, 80, 521, 136, 31, 27, 324, 175, 2, 325, 35],
 [79, 6, 5, 39, 238, 35, 80, 521, 136, 31, 27, 324, 175, 2, 325, 35, 176],
 [79, 6, 5, 39, 238, 35, 80, 521, 136, 31, 27, 324, 175, 2, 325, 35, 176, 137],
 [79,
  6,
  5,
  39,
  238,
  35,
  80,
  521,
  136,
  31,
  27,
  324,
  175,
  2,
  325,
  35,
  176,
  137,
  323],
 [79,
  6,
  5,
  39,
  238,
  35,
  80,
  521,
  136,
  31,
  27,
  324,
  175,
  2,
  325,
  35,
  176

In [23]:
max_len = max([len(x) for x in input_sequences])
max_len

196

In [24]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_input_sequences = pad_sequences(input_sequences, maxlen=max_len, padding='pre')

padded_input_sequences


array([[  0,   0,   0, ...,   0, 520,   2],
       [  0,   0,   0, ..., 520,   2, 323],
       [  0,   0,   0, ...,   0,  79,   6],
       ...,
       [  0,   0,   0, ...,   2, 119,  82],
       [  0,   0,   0, ..., 119,  82,  23],
       [  0,   0,   0, ...,  82,  23,  52]], dtype=int32)

In [25]:
X = padded_input_sequences[:, :-1]
y = padded_input_sequences[:, -1]

In [26]:
X

array([[  0,   0,   0, ...,   0,   0, 520],
       [  0,   0,   0, ...,   0, 520,   2],
       [  0,   0,   0, ...,   0,   0,  79],
       ...,
       [  0,   0,   0, ...,  65,   2, 119],
       [  0,   0,   0, ...,   2, 119,  82],
       [  0,   0,   0, ..., 119,  82,  23]], dtype=int32)

In [27]:
y

array([  2, 323,   6, ...,  82,  23,  52], dtype=int32)

In [28]:
X.shape

(3858, 195)

In [29]:
y.shape

(3858,)

In [30]:
num_classes = len(tokenizer.word_index) + 1

from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes=num_classes)

y.shape

(3858, 915)

In [31]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

model = Sequential()
model.add(Embedding(num_classes, 128, input_length=max_len - 1))
model.add(LSTM(350))
# model.add(LSTM(150))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.15, patience=3, min_lr=0.0001)
checkpoint_callback = ModelCheckpoint(filepath='best_model.h5',
                                      monitor='val_loss',
                                      save_best_only=True)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 195, 128)          117120    
                                                                 
 lstm_1 (LSTM)               (None, 350)               670600    
                                                                 
 dense_1 (Dense)             (None, 915)               321165    
                                                                 
Total params: 1108885 (4.23 MB)
Trainable params: 1108885 (4.23 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
import heapq
heap = []

class HeapCallback(ModelCheckpoint):
    def __init__(self, *args, **kwargs):
        super(HeapCallback, self).__init__(*args, **kwargs)

    def on_epoch_end(self, epoch, logs=None):
        current_val_loss = logs.get('val_loss')
        heapq.heappush(heap, (current_val_loss, epoch))
        super().on_epoch_end(epoch, logs)

heap_callback = HeapCallback(filepath='best_model.h5', monitor='val_loss', save_best_only=True)

history = model.fit(X, y, epochs=100, batch_size=32, validation_split=0.2, callbacks=[reduce_lr, heap_callback])


Epoch 1/100
97/97 [==============================] - 13s 108ms/step - loss: 6.3400 - accuracy: 0.0402 - val_loss: 6.1082 - val_accuracy: 0.0570 - lr: 0.0010
Epoch 2/100
97/97 [==============================] - 6s 58ms/step - loss: 6.0946 - accuracy: 0.0437 - val_loss: 5.8358 - val_accuracy: 0.0453 - lr: 0.0010
Epoch 3/100
97/97 [==============================] - 6s 62ms/step - loss: 5.8872 - accuracy: 0.0454 - val_loss: 5.7726 - val_accuracy: 0.0570 - lr: 0.0010
Epoch 4/100
97/97 [==============================] - 3s 31ms/step - loss: 5.7799 - accuracy: 0.0528 - val_loss: 5.7080 - val_accuracy: 0.0738 - lr: 0.0010
Epoch 5/100
97/97 [==============================] - 3s 31ms/step - loss: 5.6665 - accuracy: 0.0599 - val_loss: 5.6230 - val_accuracy: 0.0777 - lr: 0.0010
Epoch 6/100
97/97 [==============================] - 3s 28ms/step - loss: 5.5290 - accuracy: 0.0710 - val_loss: 5.5092 - val_accuracy: 0.0920 - lr: 0.0010
Epoch 7/100
97/97 [==============================] - 3s 34ms/step - 

In [33]:
history.history['accuracy']

[0.040181465446949005,
 0.04374594986438751,
 0.04536616802215576,
 0.052819184958934784,
 0.0599481537938118,
 0.07096564769744873,
 0.10045365989208221,
 0.13253402709960938,
 0.16558651626110077,
 0.2002592384815216,
 0.2401166558265686,
 0.28839921951293945,
 0.34769928455352783,
 0.41866493225097656,
 0.5022683143615723,
 0.5881400108337402,
 0.6584575772285461,
 0.7300713062286377,
 0.7861309051513672,
 0.8327932357788086,
 0.8758911490440369,
 0.90732342004776,
 0.9342190623283386,
 0.9494491219520569,
 0.9646791815757751,
 0.9701879620552063,
 0.9727802872657776,
 0.9792611598968506,
 0.9795852303504944,
 0.98088139295578,
 0.9821775555610657,
 0.987362265586853,
 0.9883344173431396,
 0.987362265586853,
 0.987362265586853,
 0.987038254737854,
 0.9880103468894958,
 0.987038254737854,
 0.9860661029815674,
 0.9880103468894958,
 0.987362265586853,
 0.987038254737854,
 0.987038254737854,
 0.9867141842842102,
 0.987362265586853,
 0.9867141842842102,
 0.987038254737854,
 0.98639017343

In [34]:
model.save('/content/drive/MyDrive/Models/NWPnew.h5')

In [35]:
from tensorflow.keras.models import load_model
model = load_model('/content/drive/MyDrive/Models/NWPnew.h5')


In [36]:
model.evaluate(X,y)

121/121 [==============================] - 1s 9ms/step - loss: 0.6731 - accuracy: 0.9181


[0.673149824142456, 0.9180922508239746]

In [52]:
import time
import numpy as np
text = "Road Safety"

# Figure out the correct maxlen
maxlen = model.layers[0].input_shape[1]  # Get maxlen from the model's input layer

for i in range(10):
  # tokenize
  token_text = tokenizer.texts_to_sequences([text])[0]
  # padding - Use the correct maxlen here
  padded_token_text = pad_sequences([token_text], maxlen=maxlen, padding='pre')
  # predict
  pos = np.argmax(model.predict(padded_token_text))

  for word,index in tokenizer.word_index.items():
    if index == pos:
      text = text + " " + word
      print(text)
      time.sleep(1)

1/1 [==============================] - 0s 20ms/step
Road Safety and
1/1 [==============================] - 0s 19ms/step
Road Safety and maintenance
1/1 [==============================] - 0s 19ms/step
Road Safety and maintenance concerns
1/1 [==============================] - 0s 20ms/step
Road Safety and maintenance concerns gathering
1/1 [==============================] - 0s 20ms/step
Road Safety and maintenance concerns gathering sensor
1/1 [==============================] - 0s 21ms/step
Road Safety and maintenance concerns gathering sensor data
1/1 [==============================] - 0s 19ms/step
Road Safety and maintenance concerns gathering sensor data from
1/1 [==============================] - 0s 19ms/step
Road Safety and maintenance concerns gathering sensor data from people
1/1 [==============================] - 0s 20ms/step
Road Safety and maintenance concerns gathering sensor data from people moving
1/1 [==============================] - 0s 28ms/step
Road Safety and maintenanc

Pothole, Safety, Camera, Road, etc

In [54]:
tokenizer.word_index

{'the': 1,
 'and': 2,
 'to': 3,
 'of': 4,
 'detection': 5,
 'pothole': 6,
 'system': 7,
 'in': 8,
 'for': 9,
 'a': 10,
 'road': 11,
 'on': 12,
 'data': 13,
 'potholes': 14,
 'maintenance': 15,
 'by': 16,
 'learning': 17,
 'with': 18,
 'real': 19,
 'time': 20,
 'or': 21,
 'can': 22,
 'transportation': 23,
 'safety': 24,
 'image': 25,
 'algorithms': 26,
 'as': 27,
 'this': 28,
 'infrastructure': 29,
 'processing': 30,
 'such': 31,
 'these': 32,
 'approach': 33,
 'research': 34,
 'camera': 35,
 'drivers': 36,
 'deep': 37,
 'be': 38,
 'systems': 39,
 'from': 40,
 'proactive': 41,
 'are': 42,
 'machine': 43,
 'cameras': 44,
 'quality': 45,
 'accuracy': 46,
 'using': 47,
 'paper': 48,
 '●': 49,
 'is': 50,
 'performance': 51,
 'efficiency': 52,
 'potentially': 53,
 'traffic': 54,
 'study': 55,
 'which': 56,
 'false': 57,
 'potential': 58,
 'identify': 59,
 'enhance': 60,
 'leveraging': 61,
 'vehicle': 62,
 'will': 63,
 'surface': 64,
 'costs': 65,
 'implementing': 66,
 'drawbacks': 67,
 'redu